# Online Semi-Supervised Learning in Contextual Bandits with Episodic Reward
에피소드 보상이 있는 컨텍스트 밴딧에서의 온라인 준지도 학습
---

## Abstract
- 실제 응용에서는 보상 피드백이 항상 주어지지 않고, 맥락(context)도 시점마다 변하는 경우가 많음.
- 이를 다루기 위해 **에피소드 보상 온라인 학습(Online Learning with Episodic Reward)** 문제를 정의.
- 제안: **Background Episodic Reward LinUCB (BerlinUCB)**
  - 보상이 없을 때도 클러스터링 기반 자기지도(self-supervision) 모듈로 컨텍스트 활용.
- 다양한 정적/비정적 환경에서 실험 → 기존 LinUCB 대비 성능 향상 확인.
- 실제 사례에서도 유용함을 보임.

---

## 1. Introduction
- 온라인 학습: 데이터가 순차적으로 들어오고, 이를 기반으로 미래 보상/예측을 업데이트하는 과정.
- 목표: **순차적 의사결정** 상황에서 매 시점 최적 행동을 선택 → **누적 보상(cumulative reward)** 최대화.
- 핵심 과제: **탐색(Exploration)** vs **활용(Exploitation)** 균형.

- **문제점**: 실제 응용에서는 보상이 항상 주어지지 않음.
  - 추천 시스템: 사용자가 반응 안 함 → 단순 0(부정)으로 볼 수 없음.
  - 의료 데이터: 환자 추적 검진 결측.
  - 광고 시스템: 특정 시간대(황금시간대)만 반응 ↑ → **비정상성(Nonstationarity)** 존재.

- **제안**:
  - 새로운 문제 정의: *Online Learning with Episodic Reward*
  - 세미-슈퍼바이즈드(semi-supervised) + 셀프-슈퍼바이즈드(self-supervised) 학습으로
    보상이 없는 데이터도 활용.

---

## 2. Related Work
- **LinUCB**
  - 대표적인 컨텍스트 밴딧 알고리즘.
  - 보상 = 컨텍스트의 선형 함수라 가정.
  - 공분산 행렬 \(A\), 보상 벡터 \(b\) 기반으로 파라미터 추정.
  - 기대 보상:
$$
E[r_t^k \mid x_t] = \mu_k^T x_t
$$

  - Upper Confidence Bound(UCB)로 탐색/활용 조절.
  - 단점: **보상이 항상 제공**된다고 가정.

- **세미-슈퍼바이즈드 학습**
  - 일부만 레이블이 있고 나머지는 없음.
  - **클러스터 가정**: 같은 클러스터 내 데이터는 같은 레이블일 가능성↑.

- **셀프-슈퍼바이즈드 학습**
  - 데이터 자체에서 감독 신호 생성.
  - 최근 비전/NLP에서 성공적 → 지도학습 수준 성능에 근접.

- **본 연구의 차별성**
  - LinUCB 기반에 세미/셀프-슈퍼바이즈드 모듈 결합.
  - 결측 보상 상황에서도 학습 지속 가능.

---



# Semi-supervised 학습이란?

- Supervised learning: 모든 데이터에 레이블(정답)이 붙어 있음 → 그걸로 학습.

- Unsupervised learning: 아예 레이블 없이 데이터 구조(클러스터, 패턴)만 학습.

- Semi-supervised learning: 일부 데이터만 레이블이 있고, 나머지는 레이블이 없음.

- 레이블 있는 데이터 → 정확한 지도 신호 제공.

- 레이블 없는 데이터 → 데이터의 구조(분포, 클러스터)를 이용해 보완.

- ## 핵심 가정(Assumptions)

- 1.Smoothness assumption: 가까운 점은 같은 레이블일 가능성이 크다.

- 2.Cluster assumption: 같은 클러스터에 속한 데이터는 같은 레이블일 가능성이 크다.

- 3.Manifold assumption: 데이터가 고차원 공간에 퍼져 있지만, 실제는 더 낮은 차원의 다양체(manifold)에 놓여 있다.

## 대표 기법

- Pseudo-labeling: 모델이 unlabeled 데이터에 임시 레이블을 붙여 다시 학습.

- Consistency regularization: 레이블 없는 데이터에 약간의 변형을 줘도 예측이 크게 달라지지 않도록 학습.

- Graph-based methods: 데이터 간 유사도를 그래프로 표현, 라벨이 있는 노드에서 없는 노드로 전파(label propagation).

- 의의

현실 데이터에서 레이블링 비용이 크거나 제한적일 때 매우 유용.

예: 의료 영상(전문가가 일일이 판독해야 하는 고비용), 추천 시스템(사용자 클릭 로그는 많지만 정답 피드백은 적음).

---


## 3. Problem Setting

### 3.1 컨텍스트 밴딧
- 시점 \(t\)에서 컨텍스트 $x_t \in \mathbb{R}^N$ 주어짐.
- 행동 \(a_t\) 선택.
- 보상 $r_t^k \in \{0,1\}$.
.
- 기대 보상은 선형 함수:
  $$
  E[r_t^k | x_t] = \mu_k^T x_t
  $$
- 여기서 $\mu_k$
는 학습해야 하는 암(arm)별 파라미터.

---

### 3.1 알고리즘 1: Online Learning with Episodic Reward
1. \(t = 1,...,T\) 반복
2. (x(t), r(t)) 샘플링
3. 컨텍스트 x(t) 공개
4. 행동 \(a_t\) 선택
5. **선택된 암의 보상만 에피소드적으로 드러남**
6. 정책 업데이트

---

### 3.2 온라인 준지도 학습
- 전통 지도학습: 매번 정답 레이블 제공.
- 본 문제: 밴딧 피드백만 존재 → “맞았다/틀렸다”만 알 수 있고 정답 레이블은 모름.
- 따라서 **레이블 있는 + 없는 데이터 모두 활용**해야 함.
- 이 문제를 **온라인 세미-슈퍼바이즈드 학습**으로 정의.

---

## 4. BerlinUCB

- **제안 방법**: Background Episodic Reward LinUCB (BerlinUCB)  
  - 준지도(semi-supervised) + 자기지도(self-supervised) 방식의 컨텍스트 밴딧.  
  - 피드백(보상)이 있을 때 / 없을 때를 구분해 서로 다른 업데이트 수행.  

- **업데이트 방식**:
  - **보상 있음** → LinUCB처럼 \(A, b\) 모두 업데이트 (정상 학습).  
  - **보상 없음** → 과거 데이터 기반 **클러스터링 모듈**(K-means, GMM, KNN)로 arm \(a'\) 추정.  
    - \(b\)만 업데이트, \(A\)는 유지.  
    - 즉, uncertainty 구조는 그대로 두고, 보상 벡터만 보정.

- **BerlinUCB의 의미**:
  - 보상이 없어도 학습 가능 → **semi-supervised bandit** 구조.  
  - 현실적 시나리오 반영:
    - 뉴스 추천: 사용자가 기사 안 눌러도, 클릭 분포/비슷한 사용자 행동으로 간접 학습 가능.  
    - 의료 실험: 환자가 즉시 반응을 보고하지 않아도, 다른 환자 데이터 패턴을 통해 업데이트 가능.  
  - 기존 LinUCB와 달리 **episodic/delayed feedback 상황에서도 성능 유지**.  
  - 실험(논문 5장)에서도 BerlinUCB 변형(B-Kmeans, B-GMM, B-KNN)이 **피드백 결측률↑ 환경에서 우수한 성능**을 보여줌.



# **BerlinUCB vs. LinUCB (차이점)**

  1. **보상 가정**  
     - **LinUCB**: 매 라운드마다 사용자가 arm(예: 추천 아이템)을 선택하면 **즉시 보상(reward)**이 주어진다고 가정.  
       - 예: 광고 클릭 여부가 바로 기록됨, 환자의 치료 효과가 바로 확인됨.  
       - 하지만 실제 환경에서는 사용자가 반응하지 않거나(outcome missing), 효과가 늦게 나타나는 경우가 많음.  
       - 따라서 보상이 없으면 LinUCB는 업데이트할 수 없음.  
     - **BerlinUCB**: 보상이 없는 경우에도 데이터 구조(클러스터링, 유사도)를 이용해 arm을 추정하고 자기지도 업데이트를 수행.  
       - 즉, **“보상이 없어도 학습을 멈추지 않는다”**는 점이 핵심.

  2. **데이터 활용성**  
     - **Labeled feedback**: 데이터 + 보상(label)이 함께 있는 경우.  
       - 예: “이 사용자가 이 영화를 보고 ★ 5점을 줌” → 라벨이 있음.  
     - **Unlabeled feedback**: 데이터는 있지만 보상(label)이 없는 경우.  
       - 예: “이 사용자가 영화를 보긴 했는데, 만족도 점수는 모름.”  
     - **LinUCB**: labeled feedback만 사용 가능 → unlabeled 데이터는 버려짐.  
     - **BerlinUCB**: labeled + unlabeled 모두 활용.  
       - 보상 없는 데이터도 클러스터 구조를 통해 보상 추정에 기여 → **semi-supervised bandit**.  

  3. **현실성**  
     - **LinUCB**: 수학적으로 단순하고 분석이 깔끔하지만, **“보상이 항상 주어진다”**는 가정이 현실과는 다름.  
       - 실제 온라인 서비스에서는 사용자가 반응하지 않는 경우가 훨씬 많음.  
       - 의료 데이터에서도 outcome을 얻는 데 시간이 걸림 → 즉시 피드백 불가능.  
     - **BerlinUCB**: 보상이 간헐적(episodic), 지연(delayed), 결측(missing)인 상황을 반영.  
       - 따라서 **추천 시스템, 의료 데이터 분석, 온라인 학습 환경**에서 더 현실적으로 적용 가능.

  4. **성능**  
     - 실험적으로도 BerlinUCB는 특히 **보상 결측률이 높은 환경**에서 LinUCB 대비 성능 우위.  
     - 이유:
       1. 보상이 있는 경우 → LinUCB와 동일하게 정확한 업데이트 수행.  
       2. 보상이 없는 경우 → unlabeled 데이터를 버리지 않고, K-means/GMM/KNN 같은 모듈로 arm을 예측해 부분 업데이트 수행.  
     - 결과적으로, LinUCB는 보상이 빠진 구간에서 학습 공백이 생기는 반면,  
       BerlinUCB는 그 구간에서도 학습을 이어가기 때문에 누적 성능 차이가 벌어짐.


---
## 4. Background Episodically Rewarded LinUCB

### 4.1 Self/Semi-supervision 모듈
- 클러스터 가정에 따라 설계.
- **K-means, GMM, K-NN** 사용.
- **K-means**: 피처 평균을 업데이트하여 arm 추정.
- **GMM**: 클러스터별 평균과 분산을 추정해 arm 예측.
- **KNN**: 가까운 이웃들의 다수결로 arm 예측.
- 온라인에서도 적용 가능.


In [ ]:
# k-means


function result = kmeansclf(data,opts)

if opts.oracle == 1
    nArms = opts.nOptions;
    labls = [];
    labln = [];
else
    nArms = 1; % 1 for the new arm
    labls = ['new'];
    labln = [1];
end

y = data.y;
y_true = data.full_y;
x = data.rec;

rew = 0;
r = [];
a = [];

% kms = zeros(nArms,data.dim);
kms = x(:,1:nArms)';
kmn = zeros(nArms,1);
idx = 1:nArms;

hbk = cell(nArms,1);
for i = 1:nArms
    hbk{i} = [];
end

for t = 1:data.t
    disp(strcat('kmeans - ',num2str(t)))

    feat = x(:,t)';
    labl = y(t);
    labl_true = y_true(t);

    stillWrong = 0;
    stillCorrect = 0;

    pred = mode(idx(knnsearch(kms,feat)));

    if opts.oracle == 1

        if ~any(labln == pred)
            if labl ~= "-1" && (isempty(labls) || ~any(labls == labl))
                labls = [labls;labl];
                labln = [labln;length(labls)];
                pred = length(labls);
            else
                stillWrong = 1;
            end
        else
            if labl ~= "-1" && (isempty(labls) || ~any(labls == labl))
                labls = [labls;labl];
                labln = [labln;length(labls)];
            end
        end

    else

        if ~any(labls == labl) && labl ~= "-1"
            nArms = nArms + 1;
            labls = [labls;labl];
            labln = [labln;nArms];
            kms = [kms;feat];
            kmn = [kmn; 0];
            idx = 1:nArms;
            hbk{nArms} = [];
            if pred == 1
                stillCorrect = 1;
            end
        end
    end

    if ~stillWrong && (stillCorrect || labls(labln == pred) == labl_true)
        rew = rew + 1;
    end

    r = [r;rew];
    acc = rew / t;
    a = [a;acc];

    if labl ~= "-1"
        assignment = labln(labls == labl);
        kmn(assignment) = kmn(assignment) + 1;
        hbk{assignment} = [hbk{assignment};feat];
        idx = 1:nArms;
        kms(assignment,:) = mean(hbk{assignment},1);
    end

end
acc = rew / data.t;

result.acc = acc;
result.rew = rew;
result.a = a;
result.r = r;

end

In [ ]:
# gmm


function result = gmmclf(data,opts)

if opts.oracle == 1
    nArms = opts.nOptions;
    labls = [];
    labln = [];
else
    nArms = 1; % 1 for the new arm
    labls = ['new'];
    labln = [1];
end

y = data.y;
y_true = data.full_y;
x = data.rec;

rew = 0;
r = [];
a = [];

% kms = zeros(nArms,data.dim);
kms = x(:,1:nArms)';
kmv = ones(nArms,data.dim);
kmn = zeros(nArms,1);
idx = 1:nArms;

hbk = cell(nArms,1);
for i = 1:nArms
    hbk{i} = [];
end

for t = 1:data.t
    disp(strcat('gmm - ',num2str(t)))

    feat = x(:,t)';
    labl = y(t);
    labl_true = y_true(t);

    stillWrong = 0;
    stillCorrect = 0;

    nlls = [];
    for i = 1:nArms
        nll = 0;
        for j = 1:data.dim
            nll = nll + normlike([kms(i,j),kmv(i,j)],feat(j));
        end
        nlls = [nlls; nll];
    end
    [snlls,scores] = sort(nll);
    pred = scores(1);

    if opts.oracle == 1

        if ~any(labln == pred)
            if labl ~= "-1" && (isempty(labls) || ~any(labls == labl))
                labls = [labls;labl];
                labln = [labln;length(labls)];
                pred = length(labls);
            else
                stillWrong = 1;
            end
        else
            if labl ~= "-1" && (isempty(labls) || ~any(labls == labl))
                labls = [labls;labl];
                labln = [labln;length(labls)];
            end
        end

    else

        if ~any(labls == labl) && labl ~= "-1"
            nArms = nArms + 1;
            labls = [labls;labl];
            labln = [labln;nArms];
            kms = [kms;feat];
            kmv = [kmv;ones(1,data.dim)];
            kmn = [kmn; 0];
            idx = 1:nArms;
            hbk{nArms} = [];
            if pred == 1
                stillCorrect = 1;
            end
        end
    end

    if ~stillWrong && (stillCorrect || labls(labln == pred) == labl_true)
        rew = rew + 1;
    end

    r = [r;rew];
    acc = rew / t;
    a = [a;acc];

    if labl ~= "-1"
        assignment = labln(labls == labl);
        kmn(assignment) = kmn(assignment) + 1;
        hbk{assignment} = [hbk{assignment};feat];
        kms(assignment,:) = mean(hbk{assignment},1);
        kmv(assignment,:) = std(hbk{assignment},1);
    end

end
acc = rew / data.t;

result.acc = acc;
result.rew = rew;
result.a = a;
result.r = r;

end

In [ ]:
# knn



function result = knnclf(data,opts)

if opts.oracle == 1
    nArms = opts.nOptions;
    labls = [];
    labln = [];
else
    nArms = 1; % 1 for the new arm
    labls = ['new'];
    labln = [1];
end

k = 5;
y = data.y;
y_true = data.full_y;
x = data.rec;

rew = 0;
r = [];
a = [];

% kms = zeros(nArms,data.dim);
kms = x(:,1:nArms)';
kmn = zeros(nArms,1);
idx = [1:nArms];

hbk = cell(nArms,1);
for i = 1:nArms
    hbk{i} = [];
end

for t = 1:data.t
    disp(strcat('knn - ',num2str(t)))

    feat = x(:,t)';
    labl = y(t);
    labl_true = y_true(t);

    stillWrong = 0;
    stillCorrect = 0;

    knns = [];
    knny = [];
    for i = 1:nArms
        knns = [knns;hbk{i}];
        knny = [knny;i*ones(kmn(i),1)];
    end
    if size(knns,1) < k+1
        pred = mode(idx(knnsearch(kms,feat)));
    else
        pred = mode(knny(knnsearch(knns,feat,'K',k)));
    end

    if opts.oracle == 1

        if ~any(labln == pred)
            if labl ~= "-1" && (isempty(labls) || ~any(labls == labl))
                labls = [labls;labl];
                labln = [labln;length(labls)];
                pred = length(labls);
            else
                stillWrong = 1;
            end
        else
            if labl ~= "-1" && (isempty(labls) || ~any(labls == labl))
                labls = [labls;labl];
                labln = [labln;length(labls)];
            end
        end

    else

        if ~any(labls == labl) && labl ~= "-1"
            nArms = nArms + 1;
            labls = [labls;labl];
            labln = [labln;nArms];
            kms = [kms;feat];
            kmn = [kmn; 0];
            idx = 1:nArms;
            hbk{nArms} = [];
            if pred == 1
                stillCorrect = 1;
            end
        end
    end

    if ~stillWrong && (stillCorrect || labls(labln == pred) == labl_true)
        rew = rew + 1;
    end

    r = [r;rew];
    acc = rew / t;
    a = [a;acc];

    if labl ~= "-1"
        assignment = labln(labls == labl);
        kmn(assignment) = kmn(assignment) + 1;
        hbk{assignment} = [hbk{assignment};feat];
        kms(assignment,:) = mean(hbk{assignment},1);
    end

end
acc = rew / data.t;

result.acc = acc;
result.rew = rew;
result.a = a;
result.r = r;

end

---

### 4.2 비교 대상

- 보상이 없는 데이터(unlabeled data)도 클러스터 구조를 활용해 arm을 추정 → 부분 업데이트 가능.

- 온라인 환경 지원: 데이터가 순차적으로 들어와도 즉시 클러스터링 적용 → 현실 시스템에서도 활용 가능.

1. **LinUCB**: 결측 피드백을 0으로 처리.
2. **BerlinUCB**: 자기지도 모듈 없음.
3. **B-Kmeans**: BerlinUCB + K-means.
4. **B-KNN**: BerlinUCB + K-NN.
5. **B-GMM**: BerlinUCB + GMM.

In [ ]:
# 1. linucb


function result = linucb(data,opts)

ucb_alpha = 0.1;
A = {};
b = {};

if opts.oracle == 1
    nArms = opts.nOptions;
    labls = [];
    labln = [];
    for i = 1:nArms
        A{i} = eye(data.dim);
        b{i} = zeros(data.dim,1);
    end
else
    nArms = 1; % 1 for the new arm
    labls = ['new'];
    labln = [1];
    A{1} = eye(data.dim);
    b{1} = zeros(data.dim,1);
end

y = data.y;
y_true = data.full_y;
x = data.rec;

rew = 0;
r = [];
a = [];

for t = 1:data.t
    disp(strcat('linucb - ',num2str(t)))

    feat = x(:,t)';
    labl = y(t);
    labl_true = y_true(t);

    stillWrong = 0;
    stillCorrect = 0;

    ps = [];
    for i = 1:nArms
        theta = A{i}{i};
        p = theta'*feat'+ucb_alpha*sqrt(feat*(A{i}
eat'));
        ps = [ps, p];
    end

    [sps,scores] = sort(-ps);
    pred = scores(1);

    if opts.oracle == 1
        if ~any(labln == pred)
            if labl ~= "-1" && (isempty(labls) || ~any(labls == labl))
                labls = [labls;labl];
                labln = [labln;length(labls)];
                pred = length(labls);
            else
                stillWrong = 1;
            end
        else
            if labl ~= "-1" && (isempty(labls) || ~any(labls == labl))
                labls = [labls;labl];
                labln = [labln;length(labls)];
            end
        end
    else
        if ~any(labls == labl) && labl ~= "-1"
            nArms = nArms + 1;
            labls = [labls;labl];
            labln = [labln;nArms];
            A{nArms} = eye(data.dim);
            b{nArms} = zeros(data.dim,1);
            if pred == 1
                stillCorrect = 1;
            end
        end
    end

    if ~stillWrong && (stillCorrect || labls(labln == pred) == labl_true)
        rew = rew + 1;
    end

    r = [r;rew];
    acc = rew / t;
    a = [a;acc];

    if labl ~= "-1"    # 보상이 있을 때만
        if any(labln == pred) && labls(labln == pred) == labl
            rt = 1;
        else
            rt = 0;
        end
        A{pred} = A{pred} + feat'*feat;
        b{pred} = b{pred} + rt*feat';
    end

end
acc = rew / data.t;

result.acc = acc;
result.rew = rew;
result.a = a;
result.r = r;

end

In [ ]:
# 2. berlinucb


function result = berlinucb(data,opts)

ucb_alpha = 0.1;
A = {};
b = {};

if opts.oracle == 1
    nArms = opts.nOptions;
    labls = [];
    labln = [];
    for i = 1:nArms
        A{i} = eye(data.dim);
        b{i} = zeros(data.dim,1);
    end
else
    nArms = 1; % 1 for the new arm
    labls = ['new'];
    labln = [1];
    A{1} = eye(data.dim);
    b{1} = zeros(data.dim,1);
end

y = data.y;
y_true = data.full_y;
x = data.rec;

rew = 0;
r = [];
a = [];

for t = 1:data.t
    disp(strcat('berlinucb - ',num2str(t)))

    feat = x(:,t)';
    labl = y(t);
    labl_true = y_true(t);

    stillWrong = 0;
    stillCorrect = 0;

    ps = [];
    for i = 1:nArms
        theta = A{i}{i};
        p = theta'*feat'+ucb_alpha*sqrt(feat*(A{i}
eat'));
        ps = [ps, p];
    end

    [sps,scores] = sort(-ps);
    pred = scores(1);

    if opts.oracle == 1
        if ~any(labln == pred)
            if labl ~= "-1" && (isempty(labls) || ~any(labls == labl))
                labls = [labls;labl];
                labln = [labln;length(labls)];
                pred = length(labls);
            else
                stillWrong = 1;
            end
        else
            if labl ~= "-1" && (isempty(labls) || ~any(labls == labl))
                labls = [labls;labl];
                labln = [labln;length(labls)];
            end
        end
    else
        if ~any(labls == labl) && labl ~= "-1"
            nArms = nArms + 1;
            labls = [labls;labl];
            labln = [labln;nArms];
            A{nArms} = eye(data.dim);
            b{nArms} = zeros(data.dim,1);
            if pred == 1
                stillCorrect = 1;
            end
        end
    end

    if ~stillWrong && (stillCorrect || labls(labln == pred) == labl_true)
        rew = rew + 1;
    end

    r = [r;rew];
    acc = rew / t;
    a = [a;acc];

    if labl ~= "-1" && any(labln == pred) && labls(labln == pred) == labl
        rt = 1;
    else
        rt = 0;
    end
    A{pred} = A{pred} + feat'*feat;
    b{pred} = b{pred} + rt*feat';


end
acc = rew / data.t;

result.acc = acc;
result.rew = rew;
result.a = a;
result.r = r;

end

In [ ]:
# 3. berlinucbkmeans



function result = berlinucbkmeans(data,opts)

ucb_alpha = 0.1;
A = {};
b = {};

if opts.oracle == 1
    nArms = opts.nOptions;
    labls = [];
    labln = [];
    for i = 1:nArms
        A{i} = eye(data.dim);
        b{i} = zeros(data.dim,1);
    end
else
    nArms = 1; % 1 for the new arm
    labls = ['new'];
    labln = [1];
    A{1} = eye(data.dim);
    b{1} = zeros(data.dim,1);
end

y = data.y;
y_true = data.full_y;
x = data.rec;

rew = 0;
r = [];
a = [];

% kms = zeros(nArms,data.dim);
kms = x(:,1:nArms)';
kmn = zeros(nArms,1);
idx = 1:nArms;

hbk = cell(nArms,1);
for i = 1:nArms
    hbk{i} = [];
end

for t = 1:data.t
    disp(strcat('berlinucbkmeans - ',num2str(t)))
    feat = x(:,t)';
    labl = y(t);
    labl_true = y_true(t);

    stillWrong = 0;
    stillCorrect = 0;

    ps = [];
    for i = 1:nArms
        theta = A{i}{i};
        p = theta'*feat'+ucb_alpha*sqrt(feat*(A{i}
eat'));
        ps = [ps, p];
    end
    [sps,scores] = sort(-ps);
    pred = scores(1);

    if opts.oracle == 1
        if ~any(labln == pred)
            if labl ~= "-1" && (isempty(labls) || ~any(labls == labl))
                labls = [labls;labl];
                labln = [labln;length(labls)];
                pred = length(labls);
            else
                stillWrong = 1;
            end
        else
            if labl ~= "-1" && (isempty(labls) || ~any(labls == labl))
                labls = [labls;labl];
                labln = [labln;length(labls)];
            end
        end
    else
        if ~any(labls == labl) && labl ~= "-1"
            nArms = nArms + 1;
            labls = [labls;labl];
            labln = [labln;nArms];
            kms = [kms;feat];
            kmn = [kmn; 0];
            idx = 1:nArms;
            hbk{nArms} = [];
            A{nArms} = eye(data.dim);
            b{nArms} = zeros(data.dim,1);
            if pred == 1
                stillCorrect = 1;
            end
        end
    end

    if ~stillWrong && (stillCorrect || labls(labln == pred) == labl_true)
        rew = rew + 1;
    end

    r = [r;rew];
    acc = rew / t;
    a = [a;acc];

    if (labl == "-1") #보상이 없을 때

        pred_self = mode(idx(knnsearch(kms,feat)));
        if pred_self == pred
            rt = 1;
        else
            rt = 0;
        end
%          A{pred} = A{pred} + feat'*feat;
        b{pred} = b{pred} + rt*feat';

    else

        assignment = labln(labls == labl);
        kmn(assignment) = kmn(assignment) + 1;
        hbk{assignment} = [hbk{assignment};feat];
        kms(assignment,:) = mean(hbk{assignment},1);

        if labls(labln == pred) == labl
            rt = 1;
        else
            rt = 0;
        end
        A{pred} = A{pred} + feat'*feat;
        b{pred} = b{pred} + rt*feat';

    end

end
acc = rew / data.t;

result.acc = acc;
result.rew = rew;
result.a = a;
result.r = r;

end

In [ ]:
# 4. berlinucbknn



function result = berlinucbknn(data,opts)

ucb_alpha = 0.1;
A = {};
b = {};
k = 5;

if opts.oracle == 1
    nArms = opts.nOptions;
    labls = [];
    labln = [];
    for i = 1:nArms
        A{i} = eye(data.dim);
        b{i} = zeros(data.dim,1);
    end
else
    nArms = 1; % 1 for the new arm
    labls = ['new'];
    labln = [1];
    A{1} = eye(data.dim);
    b{1} = zeros(data.dim,1);
end

y = data.y;
y_true = data.full_y;
x = data.rec;

rew = 0;
r = [];
a = [];

% kms = zeros(nArms,data.dim);
kms = x(:,1:nArms)';
kmn = zeros(nArms,1);
idx = 1:nArms;

hbk = cell(nArms,1);
for i = 1:nArms
    hbk{i} = [];
end

for t = 1:data.t
    disp(strcat('berlinucbknn - ',num2str(t)))
    feat = x(:,t)';
    labl = y(t);
    labl_true = y_true(t);

    stillWrong = 0;
    stillCorrect = 0;

    ps = [];
    for i = 1:nArms
        theta = A{i}{i};
        p = theta'*feat'+ucb_alpha*sqrt(feat*(A{i}
eat'));
        ps = [ps, p];
    end
    [sps,scores] = sort(-ps);
    pred = scores(1);

    if opts.oracle == 1
        if ~any(labln == pred)
            if labl ~= "-1" && (isempty(labls) || ~any(labls == labl))
                labls = [labls;labl];
                labln = [labln;length(labls)];
                pred = length(labls);
            else
                stillWrong = 1;
            end
        else
            if labl ~= "-1" && (isempty(labls) || ~any(labls == labl))
                labls = [labls;labl];
                labln = [labln;length(labls)];
            end
        end
    else
        if ~any(labls == labl) && labl ~= "-1"
            nArms = nArms + 1;
            labls = [labls;labl];
            labln = [labln;nArms];
            kms = [kms;feat];
            kmn = [kmn; 0];
            idx = 1:nArms;
            hbk{nArms} = [];
            A{nArms} = eye(data.dim);
            b{nArms} = zeros(data.dim,1);
            if pred == 1
                stillCorrect = 1;
            end
        end
    end

    if ~stillWrong && (stillCorrect || labls(labln == pred) == labl_true)
        rew = rew + 1;
    end

    r = [r;rew];
    acc = rew / t;
    a = [a;acc];

    if (labl == "-1") #보상이 없을 때

        knns = [];
        knny = [];
        for i = 1:nArms
            knns = [knns;hbk{i}];       #knns: 지금까지 쌓인 feature 히스토리
            knny = [knny;i*ones(kmn(i),1)];  #knny: 각 feature의 arm 라벨 기록
        end
        if size(knns,1) < k+1
            pred_self = mode(idx(knnsearch(kms,feat)));
        else
            pred_self = mode(knny(knnsearch(knns,feat,'K',k)));
        end
        if pred_self == pred
            rt = 1;
        else
            rt = 0;
        end
        %          A{pred} = A{pred} + feat'*feat;
        b{pred} = b{pred} + rt*feat';

    else

        assignment = labln(labls == labl);
        kmn(assignment) = kmn(assignment) + 1;
        hbk{assignment} = [hbk{assignment};feat];
        kms(assignment,:) = mean(hbk{assignment},1);

        if labls(labln == pred) == labl
            rt = 1;
        else
            rt = 0;
        end
        A{pred} = A{pred} + feat'*feat;
        b{pred} = b{pred} + rt*feat';

    end

end
acc = rew / data.t;

result.acc = acc;
result.rew = rew;
result.a = a;
result.r = r;

end

In [ ]:
# 5. berlinucbgmm

function result = berlinucbgmm(data,opts)

ucb_alpha = 0.1;
A = {};
b = {};

if opts.oracle == 1
    nArms = opts.nOptions;
    labls = [];
    labln = [];
    for i = 1:nArms
        A{i} = eye(data.dim);
        b{i} = zeros(data.dim,1);
    end
else
    nArms = 1; % 1 for the new arm
    labls = ['new'];
    labln = [1];
    A{1} = eye(data.dim);
    b{1} = zeros(data.dim,1);
end

y = data.y;
y_true = data.full_y;
x = data.rec;

rew = 0;
r = [];
a = [];

% kms = zeros(nArms,data.dim);
kms = x(:,1:nArms)';
kmv = ones(nArms,data.dim);
kmn = zeros(nArms,1);
idx = 1:nArms;

hbk = cell(nArms,1);
for i = 1:nArms
    hbk{i} = [];
end

for t = 1:data.t
    disp(strcat('berlinucbgmm - ',num2str(t)))
    feat = x(:,t)';
    labl = y(t);
    labl_true = y_true(t);

    stillWrong = 0;
    stillCorrect = 0;

    ps = [];
    for i = 1:nArms
        theta = A{i}{i};
        p = theta'*feat'+ucb_alpha*sqrt(feat*(A{i}
eat'));
        ps = [ps, p];
    end
    [sps,scores] = sort(-ps);
    pred = scores(1);

    if opts.oracle == 1
        if ~any(labln == pred)
            if labl ~= "-1" && (isempty(labls) || ~any(labls == labl))
                labls = [labls;labl];
                labln = [labln;length(labls)];
                pred = length(labls);
            else
                stillWrong = 1;
            end
        else
            if labl ~= "-1" && (isempty(labls) || ~any(labls == labl))
                labls = [labls;labl];
                labln = [labln;length(labls)];
            end
        end
    else
        if ~any(labls == labl) && labl ~= "-1"
            nArms = nArms + 1;
            labls = [labls;labl];
            labln = [labln;nArms];
            kms = [kms;feat];
            kmv = [kmv;ones(1,data.dim)];
            kmn = [kmn; 0];
            idx = 1:nArms;
            hbk{nArms} = [];
            A{nArms} = eye(data.dim);
            b{nArms} = zeros(data.dim,1);
            if pred == 1
                stillCorrect = 1;
            end
        end
    end

    if ~stillWrong && (stillCorrect || labls(labln == pred) == labl_true)
        rew = rew + 1;
    end

    r = [r;rew];
    acc = rew / t;
    a = [a;acc];

    if (labl == "-1")  # 보상이 없을 때

        nlls = [];
        for i = 1:nArms
            nll = 0;
            for j = 1:data.dim
                nll = nll + normlike([kms(i,j),kmv(i,j)],feat(j)); #음의 로그우도
            end
            nlls = [nlls; nll];
        end
        [snlls,scores] = sort(nll);
        pred_self = scores(1);  # 가장 가능도 높은 arm
        if pred_self == pred
            rt = 1;
        else
            rt = 0;
        end
        %          A{pred} = A{pred} + feat'*feat;
        b{pred} = b{pred} + rt*feat';

    else

        assignment = labln(labls == labl);
        kmn(assignment) = kmn(assignment) + 1;
        hbk{assignment} = [hbk{assignment};feat];
        kms(assignment,:) = mean(hbk{assignment},1);
        kmv(assignment,:) = std(hbk{assignment},1);

        if labls(labln == pred) == labl
            rt = 1;
        else
            rt = 0;
        end
        A{pred} = A{pred} + feat'*feat;
        b{pred} = b{pred} + rt*feat';

    end

end
acc = rew / data.t;

result.acc = acc;
result.rew = rew;
result.a = a;
result.r = r;

end

## 5. Experiments

### 5.1 실험 목적
- 제안한 **BerlinUCB**와 변형들(B-Kmeans, B-KNN, B-GMM)을 기존 **LinUCB**와 비교.
- 평가 목표:
  1. 보상이 드러나지 않을 때도 학습 성능 유지 여부.
  2. 자기지도(self-supervision) 모듈이 성능 향상에 기여하는지.

---

### 5.2 데이터셋
- **Stationary 환경**
  - **MNIST**: 손글씨 숫자 데이터 (정적 환경에서의 실험).
  - **Warfarin dataset**: 환자 특성(성별, 연령, 인구통계 등)을 기반으로 약물 용량 결정.

- **Non-stationary 환경**
  - 데이터 분포와 보상 분포가 시간이 지남에 따라 변화하는 환경 설계.
  - 다양한 시나리오(총 6가지)에서 실험.

---

### 5.3 설정
- 각 실험은 **T = 10,000 라운드** 진행.
- 보상 피드백은 확률 \(p_r\)로만 드러남:
  $ p_r \in \{0.25, 0.5, 0.75\} $




- 피드백이 없는 경우:
  - LinUCB: 보상 = 0으로 처리.
  - BerlinUCB 계열: 클러스터링 기반 자기지도 모듈 활용.

---

### 5.4 결과
- **Stationary (정적 환경)**
  - BerlinUCB가 LinUCB보다 일관적으로 높은 누적 보상.
  - 자기지도 모듈 추가(B-Kmeans, B-GMM, B-KNN) → 추가 성능 향상.
  - 특히 B-GMM이 가장 안정적.

- **Non-stationary (비정적 환경)**
  - LinUCB는 분포가 변하면 성능 급락.
  - BerlinUCB는 변화에 적응 가능.
  - 클러스터링 기반 변형(B-Kmeans, B-GMM, B-KNN)이 더 나은 적응 성능.

- 결론:  
  - **보상이 드러나지 않는 상황에서도 학습 지속 가능**.  
  - **자기지도 모듈이 추가되면 성능 향상 효과가 뚜렷함.**

---

## 6. 실제 사례 적용 (Case Study)

- 실제 응용 예시: **추천 시스템**
  - 사용자의 클릭/비클릭 로그는 보상 결측을 자주 초래.
  - BerlinUCB 구조는 이런 상황에서 **비활성 데이터를 단순히 0(부정)**으로 보지 않고,
    **잠재적 긍정/부정 혼합 신호**로 활용 가능.
  - 따라서 실제 산업 응용에서도 유용성이 높음.

---

## 7. 결론 (Conclusion)

- 본 논문은 **에피소드 보상이 있는 온라인 컨텍스트 밴딧 문제**를 정의.
- 제안: **BerlinUCB**
  - 세미-슈퍼바이즈드 + 셀프-슈퍼바이즈드 결합.
  - 보상 결측 시에도 학습 지속 가능.
- 다양한 정적/비정적 환경, 실제 사례에서 성능 향상 확인.
- 향후 과제:
  - 더 복잡한 자기지도 모듈 설계.
  - 보상 결측 메커니즘이 비정상적으로 작동하는 경우에 대한 대응.
